# EXECUTE THE CELL BELOW ONLY IF YOU'RE WORKING IN GOOGLE COLAB! 

In [ ]:
# Execute the cell only ONCE

!git clone https://gitflic.ru/project/aidynamicaction/classedu2023-advctrl.git
%cd /content/classedu2023-advctrl/assignments/asgn-1

# Introduction
In this assignment, you will learn how to design a controller that stabilizes a 3 wheel robot using a machinery of Lyapunov functions.
# Background

## Definition of Lyapunov Function
Before we dive into the details of the controller, let's first recall what Lyapunov function is. 
A Lyapunov function is a mathematical tool used in control theory and dynamical systems to analyze the stability of a system. It is a scalar function that is typically defined over the state space of the system, and it is used to assess whether the system's behavior is stable or not. More precisely,
A function $L(x)$ is a Lyapunov function for a system $\dot{x} = f(x)$ if it satisfies the following conditions:
* $L(x)$ is positive-definite, i.e., $L(x) > 0$ for all $x$ in the domain of $L(x)$, except possibly at $x = 0$.
* $L(x)$ is radially unbounded, i.e., $L(x) \rightarrow \infty$ as $|x| \rightarrow \infty$.
* $\frac{dL}{dt} = \frac{\partial L}{\partial x} f(x)$ is negative-definite, i.e., $\frac{dL}{dt} < 0$ for all $x$ in the domain of $L(x)$ except possibly at $x = 0$.

The last property is the most crucial one as it states that a Lyapunov function decays along stable trajectories. In general, it's hard to derive a Lyapunov function for an arbitrary system but we are in a more privileged position. 

## Definition of 3 wheel robot controlled system
It appeares that our 3 wheel robot represented by a controlled system 
$$
\begin{cases}
    \dot x & = v \cos \alpha  \\
    \dot y & = v \sin \alpha \\
    \dot \alpha & = \omega
\end{cases}
\quad 
\text{where $v$ and $\omega$ are control inputs}
$$
can be reduced to another system with known Lyapunov function, a so-called Brockett integrator, for which we already know a Lyapunov function (which is not the case in general!). This is a dynamical system of the form:
$$
\begin{cases}
\dot{\tilde{x}}_1=u_1 \\
\dot{\tilde{x}}_2=u_2 \\
\dot{\tilde{x}}_3=\tilde{x}_1 u_2-\tilde{x}_2 u_1
\end{cases}
\quad 
\text{where $u_1$ and $u_2$ are control inputs}
$$
We can turn our 3 wheel robot into a Brockett integrator applying the following coordinates transformation:

$$
\begin{aligned}
\tilde{\boldsymbol x}=\Psi\left(\boldsymbol x\right)= \left(\begin{array}{c}\alpha \\ x \cos \alpha+y \sin \alpha \\ -2(y \cos \alpha-x \sin \alpha)-\alpha(x \cos \alpha+y \sin \alpha) \\\end{array}\right)\end{aligned}
$$.
 
Thus, if we obtain a stabilizing action for a Brockett integrator, we can derive a corresponding stabilizing action for 3 wheel robot as well. We provide a Lyapunov function for Brockett integrator further.

**The goal of the controller is to stabilize the robot into the origin:** $\boldsymbol x_{\text{origin}} := (0, 0, 0)$

# Controller Design
The optimization-based controller works by choosing an action that minimizes a Lyapunov function of the Brockett integrator given the predicted step. 
Let's formulate the problem as an optimization problem. It can be written as:

$\boldsymbol u = \underset{\boldsymbol u^{*}}{\mathrm{argmin}} \quad L(\tilde{\boldsymbol x}_{u^{*}}^{+})$,

where $\tilde{\boldsymbol x}_{u^{*}}^{+}$ is the state vector corresponding to the Brockett integrator system in which we, $u$ is the control input, and $L(\cdot)$ is the Lyapunov function of the Brockett integrator. The objective of the optimization problem is to find the control input $u$ that minimizes the Lyapunov function $L( \tilde{\boldsymbol x}) = \tilde{x}_1^2+\tilde{x}_2^2+2 \tilde{x}_3^2+\left|\tilde{x}_3\right|\left(10-2\left(\left|\tilde{x}_1\right|+\left|\tilde{x}_2\right|\right)\right)$ given the next state $\tilde{\boldsymbol x}_{u^{*}}^{+}$.

To summarize: 

**for a given state of the 3 wheel robot** $\boldsymbol x$ **a desired action will be a solution of the following optimization problem:**
$$\boldsymbol u = \underset{\boldsymbol u^{*}}{\mathrm{argmin}} \quad L(\Psi(\boldsymbol x_{u^{*}}^{+}))$$

In [ ]:
# Execute the cell only ONCE
# Restart runtime RIGHT AFTER the cell is finished. It may fail with error --- do not mind
%pip install -r requirements.txt

In [ ]:
# EXECUTE THE CELL ONLY IF YOU'RE WORKING IN GOOGLE COLAB! 
%cd /content/classedu2023-advctrl/assignments/asgn-1

In [ ]:
from src.rcognita.__utilities import rc, Clock
from src.rcognita.systems import System
from src.rcognita.controllers import Controller
from src.rcognita.objectives import RunningObjective
from src.rcognita.simulator import Simulator
from src.rcognita.models import ModelQuadForm
from src.rcognita.scenarios import EpisodicScenarioMultirun
from src.rcognita.optimizers import SciPyOptimizer
from src.rcognita.visualization.vis_3wrobot import Animator3WRobotNI
from src import rcognita
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import rc as rc_params
rc_params('animation', html='jshtml')

## 3 wheel robot system
Complete the code for class of system to fill `Dstate` vector inside `compute_dynamics` method.

$$
\begin{cases}
    \dot x & = v \cos \alpha  \\
    \dot y & = v \sin \alpha \\
    \dot \alpha & = \omega
\end{cases}
\quad 
\text{where $v$ and $\omega$ are control inputs}
$$
In your code assume that 
1. `(action[0], action[1])` = $(v,\omega)$
2. `(state[0], state[1], state[2])` = $(x, y, \alpha)$
3. `(Dstate[0], Dstate[1], Dstate[2])` = $(\dot x, \dot y, \dot \alpha)$

In [ ]:
class Sys3WRobotNI(System):
    """
    System class: 3-wheel robot with static actuators (the NI - non-holonomic integrator).

    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.name = "3wrobotNI"

    def compute_dynamics(self, time, state, action, disturb=None):
        """
        Compute the dynamics of the system.
        """

        Dstate = np.zeros(self.dim_state)
        ####################
        #YOUR CODE GOES HERE
        ####################
        Dstate[0] = ...
        ####################
        #YOUR CODE ENDS HERE
        ####################
        return Dstate

    def out(self, state, time=None, action=None):

        return state

-------------------
## Coordinate transformation
-------------------
Implement the following coordinate transformation:

$
\begin{aligned}
\tilde{\boldsymbol x}=\Psi\left(\boldsymbol x\right)= \left(\begin{array}{c}\alpha \\ x \cos \alpha+y \sin \alpha \\ -2(y \cos \alpha-x \sin \alpha)-\alpha(x \cos \alpha+y \sin \alpha) \\\end{array}\right)\end{aligned}
$

-------------------

**Input**: `coords_Cart` - a vector of cartesian coordinates of a robot

**Output**: `xNI` - a vector of transformed coordinates


In [ ]:
def Cart2NH(coords_Cart):
        """
        Transformation from Cartesian coordinates to Brockett integrator coordinates.

        """

        xNI = np.zeros(3)
        ####################
        #YOUR CODE GOES HERE
        ####################
        
        xNI[0] = ...
        ####################
        #YOUR CODE ENDS HERE
        ####################

        return xNI

## Lyapunov function

Implement the following function:

$L(\Psi(\boldsymbol x)) = \tilde{x}_1^2+\tilde{x}_2^2+2 \tilde{x}_3^2+\left|\tilde{x}_3\right|\left(10-2\left(\left|\tilde{x}_1\right|+\left|\tilde{x}_2\right|\right)\right)$
that computes a value of a Lyapunov function given the state of a robot $\boldsymbol x$

In [ ]:
def compute_LF(state):
        
        ####################
        #YOUR CODE GOES HERE
        ####################
        
        ####################
        #YOUR CODE ENDS HERE
        ####################

        return LF_value

# Prediction

----------------------

A natural way of predicting the next state given the current action and current state is to integrate our system numerically using suitable integration method.

One of the simplest numerical integration methods is the Euler method.

Euler Method
The Euler method is a first-order numerical integration method for approximating the solution of an ODE. It uses the derivative of the solution at a given point to approximate the solution at the next point. The method is based on the following formula:

$y_{n+1} = y_n + hf(t_n,y_n)$


where $y_n$ is the approximation of the solution at time $t_n$, $h$ is the time step, $f(t_n,y_n)$ is the derivative of the solution at time $t_n$ and $y_{n+1}$ is the approximation of the solution at time $t_{n+1}=t_n+h$.

The Euler method is a first-order method, which means that its error is proportional to the size of the time step. Therefore, it may not be accurate enough for some applications. However, it is simple and easy to implement.

## Euler predictor

------------------

Fill the blank space with your code implementing Euler predictor

variables available through `self` invocation, that you may find useful:

- `self.system` - an instance of a system you've just dealt with
- `self.pred_step_size` - a size of step prediction (an $h$ from the formula of the Euler method)
- `self.compute_dynamics` - a function that you've just implemented. It computes right-hand-side of the ODE (i.e. $f(t_n,y_n)$ for Euler predictor)

In [ ]:
class EulerPredictor:
    """
    Euler predictor uses a simple Euler discretization scheme.
    It does predictions by increments scaled by a sampling time times the velocity evaluated at each successive node.

    """

    def __init__(
        self,
        pred_step_size: float,
        system: System,
        dim_input: int,
        prediction_horizon: int,
    ):
        self.system = system
        self.pred_step_size = pred_step_size
        self.compute_dynamics = system.compute_dynamics
        self.sys_out = system.out
        self.dim_input = dim_input
        self.prediction_horizon = prediction_horizon

    def predict(self, current_state, action):
        ####################
        #YOUR CODE GOES HERE
        ####################

        ####################
        #YOUR CODE ENDS HERE
        ####################
        return next_state

## Compute action

---------

Implement a method called `compute_action`, which computes an action given the observation solving the following optimization problem:

$\boldsymbol u = \underset{\boldsymbol u^{*}}{\mathrm{argmin}} \quad L(\tilde{\boldsymbol x}_{u^{*}}^{+})$

You can use an embedded optimizer for this task. Use it in the following way:

```python
action = self.optimize(objective, initial_guess, action_bounds)
```
where 
1. `objective` is a callable Python object such that `objective(`$u^*$`)`$=L(\tilde{\boldsymbol x}_{u^{*}}^{+})$.
2. `initial_guess`$=u^*_0$. Action that optimizer starts the optimization procedure with.
3. `action_bounds`. See the code of the constructor below.

*Important notice*: assign obtained solution to `self.action` in order to make everything work properly.

In [ ]:
class ControllerCLF:
    def __init__(
        self,
        action_bounds,
        optimizer,
        predictor,
        time_start=0,
        sampling_time=0.01,
    ):
        if action_bounds is None:
            action_bounds = []
        self.action_sequence_min = action_bounds[:, 0]
        self.action_sequence_max = action_bounds[:, 1]
        self.action_bounds = np.array(
            [self.action_sequence_min, self.action_sequence_max]
        )
        self.optimizer = optimizer
        self.predictor = predictor
        self.action = rc.zeros(2)
        self.clock = Clock(period=sampling_time, time_start=time_start)
        self.time_start = time_start
        self.action_old = rc.zeros(2)
        self.sampling_time = sampling_time
        self.LF = 0

    def compute_action_sampled(self, time, state, observation, observation_target=[]):
        """
        Compute sampled action.

        """

        is_time_for_new_sample = self.clock.check_time(time)

        if is_time_for_new_sample: 
            self.controller_clock = time

            action = self.compute_action(state, observation)
            self.action_old = action
            return action

        else:
            return self.action_old

    def compute_action(self, state, observation, time=0, observation_target=[]):

        ####################
        #YOUR CODE GOES HERE
        ####################

        ####################
        #YOUR CODE ENDS HERE
        ####################
        
        return self.action

## Testing!

Let's see your controller in action. A procedure below is like a LEGO constructor - we just define objects and connect it together. 

Everything here is self-evident, except, maybe `Scenario`, which is basically a class which implements a scenario of simulation itself. Step-by-step our simulator samples new state of the system and controller computes action when it's necessary to do. And all this procedure is being performed inside `scenatio.run` method.

In [ ]:
model_of_running_objective = ModelQuadForm(weights=np.diag([1, 10, 1, 0, 0]))
system = Sys3WRobotNI(
    sys_type="diff_eqn",
    dim_input=2,
    dim_output=3,
    dim_state=3,
    dim_disturb=3,
    pars=[1, 1]
    )
running_objective = RunningObjective(
    model=model_of_running_objective
    )

state_init=np.array([5., 5., 3. * np.pi / 4.])
action_init=np.ones(2)

simulator = Simulator(
    system=system,
    time_final=3,
    sys_type=system.sys_type,
    state_init=state_init,
    action_init=action_init,
    disturb_init=None,
    time_start=0,
    sampling_time=0.01,
    max_step=0.001,
    first_step=1.0E-6,
    atol=1.0E-5,
    rtol=1.0E-3,
    ode_backend="SCIPY"
)
optimizer = SciPyOptimizer(
    opt_options={
        "maxiter": 100,
        "maxfev": 5000,
        "disp": False,
        "adaptive": True,
        "xatol": 1.0E-7,
        "fatol": 1.0E-7,
    },
    opt_method="SLSQP"
)

controller = ControllerCLF(
    action_bounds=np.array([[-25, 25], [-5, 5]]),
    sampling_time=0.01,
    optimizer=optimizer,
    predictor=EulerPredictor(
        pred_step_size=0.01,
        system=system,
        dim_input=2,
        prediction_horizon=1
    )
)
scenario = EpisodicScenarioMultirun(
    running_objective=running_objective,
    simulator=simulator,
    no_print=False,
    is_log=False,
    howanim="html",
    state_init=state_init,
    action_init=action_init,
    N_episodes=1,
    N_iterations=1,
    speedup=75,
    repeat_num=0,
    controller=controller,
    observation_target=[]
)

animator = Animator3WRobotNI(
    scenario=scenario,
    fps=10,
    max_video_length=60,
    animation_max_size_mb=200
)

In [ ]:
%%capture
scenario.run() # <--- takes some time. In TA's solution not more that half the minute
animation = animator.get_animation()

In [ ]:
# This cell needs some time to render the animation (11 seconds with fps=10)
# If you want to speed up the process decrease the fps parameter in animator object

animation